In [1]:
import sys
sys.path.append('..')
from osp import *

In [2]:
df=get_pmla_df()

100%|██████████| 71902/71902 [00:03<00:00, 21703.64it/s]


In [3]:
df.decade.value_counts()

decade
2000    3104
2010    2813
1970    2556
1990    2541
1980    2513
1950    2172
1960    2106
1930    2105
1940    1956
1920    1605
1910    1233
1900     639
Name: count, dtype: int64

In [4]:
def iter_pmla_sample():
    df = get_pmla_df().sample(frac=1)
    decs = list(df.groupby('decade'))
    maxn = max(len(decdf) for dec,decdf in decs)
    for i in range(maxn):
        for dec,decdf in decs:
            if len(decdf) > i:
                yield decdf.iloc[i]

In [5]:
row = next(iter_pmla_sample())

100%|██████████| 71902/71902 [00:03<00:00, 20256.08it/s]


In [6]:
def parse_pmla_row(row, force=False):
    id = row['id']
    if not force and id in PMLA_STASH:
        return stanza.Document.from_serialized(PMLA_STASH[id])
    
    text = row['fullText']
    article = filter_pmla_pages(text)
    doc = get_nlp_doc(article)
    PMLA_STASH[id] = doc.to_serialized()
    return doc

In [7]:
doc = parse_pmla_row(row)
tree_stats = pd.DataFrame(get_tree_stats(sent) for sent in doc.sentences)
tree_stats

,num_words,height,num_independent_clauses,num_dependent_clauses,num_words_in_dependent_clauses,num_words_in_independent_clauses,num_punct,num_punct_colon,num_punct_comma,num_parens,num_words_noun,num_words_verb,num_words_adjective,num_words_adverb
0,31,18,2,1,23,8,1,0,0,0,8.0,7.0,2.0,1.0
1,73,39,1,4,183,-110,2,0,1,0,22.0,11.0,5.0,1.0
2,17,12,1,1,7,10,2,0,1,0,3.0,2.0,3.0,3.0
3,18,7,1,0,0,18,4,1,1,2,6.0,3.0,1.0,NaN
4,62,19,3,4,93,-31,7,1,5,0,12.0,7.0,3.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151,26,9,5,0,0,26,3,1,1,0,6.0,4.0,1.0,2.0
152,18,12,1,1,13,5,1,0,0,0,7.0,3.0,2.0,1.0
153,11,10,1,1,4,7,1,0,0,0,1.0,2.0,2.0,3.0
154,72,21,2,3,84,-12,5,0,4,0,17.0,9.0,7.0,5.0


In [8]:
tree_stats.mean()

num_words                           35.814103
height                              12.807692
num_independent_clauses              1.647436
num_dependent_clauses                0.871795
num_words_in_dependent_clauses      14.884615
num_words_in_independent_clauses    20.929487
num_punct                            4.846154
num_punct_colon                      0.384615
num_punct_comma                      2.435897
num_parens                           1.025641
num_words_noun                       9.536913
num_words_verb                       3.934783
num_words_adjective                  2.233645
num_words_adverb                     1.911111
dtype: float64

In [9]:
done = set(PMLA_STASH.keys_l())
len(done)

2746

In [ ]:
sample = [row for row in iter_pmla_sample() if row['id'] not in done]
len(sample)

100%|██████████| 71902/71902 [00:03<00:00, 19880.95it/s]


22597

: 

In [ ]:
for row in tqdm(sample):
    parse_pmla_row(row)

  8%|▊         | 1913/22597 [27:32:30<268:40:49, 46.76s/it] 